In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2.extras import execute_values
import pandas as pd
from glob import glob

In [ ]:
# Connection details
USER = "postgres"
PASSWORD = "root"
HOST = "localhost"
PORT = "5432"

# Step 1: Connect to default "postgres" database
conn = psycopg2.connect(
    dbname="postgres", user=USER, password=PASSWORD, host=HOST, port=PORT
)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = conn.cursor()

In [ ]:
# Step 2: Create bank_reviews database
cur.execute("SELECT 1 FROM pg_database WHERE datname = 'bank_reviews'")
exists = cur.fetchone()
if not exists:
    cur.execute("CREATE DATABASE bank_reviews")
    print("✅ Database 'bank_reviews' created.")
else:
    print("ℹ️ Database 'bank_reviews' already exists.")

cur.close()
conn.close()

In [ ]:
# Step 3: Connect to "bank_reviews" and create schema

conn = psycopg2.connect(
    dbname="bank_reviews", user=USER, password=PASSWORD, host=HOST, port=PORT
)
cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS banks (
        id SERIAL PRIMARY KEY,
        name TEXT UNIQUE NOT NULL
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS reviews (
        id SERIAL PRIMARY KEY,
        review TEXT,
        rating INTEGER,
        review_date DATE,
        bank_id INTEGER REFERENCES banks(id),
        source TEXT,
        sentiment TEXT,
        sentiment_score FLOAT
    );
""")

conn.commit()
print("✅ Tables 'banks' and 'reviews' created in 'bank_reviews'.")

cur.close()
conn.close()

In [ ]:
# Load and concatenate all CSV files (adjust path pattern if needed)
csv_files = glob("../data/processed/*.csv")  # or use absolute paths if stored elsewhere
df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# df = df.rename(columns={
#     "Date": "date",
#     "Review Text": "review",
#     "Rating": "rating",
#     "Bank/App Name": "bank",
#     "Source": "source"
# })

# Ensure date column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Preview combined data
print(df.head())
print(f"📊 Total number of rows: {len(df)}")

In [ ]:
# Step 2: Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",
    password="root",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Step 3: Insert unique banks and get their IDs
bank_names = df['bank'].unique()
bank_id_map = {}

for bank in bank_names:
    cur.execute("""
        INSERT INTO banks (name)
        VALUES (%s)
        ON CONFLICT (name) DO NOTHING
        RETURNING id;
    """, (bank,))
    result = cur.fetchone()
    if result:
        bank_id_map[bank] = result[0]
    else:
        cur.execute("SELECT id FROM banks WHERE name = %s;", (bank,))
        bank_id_map[bank] = cur.fetchone()[0]

# Step 4: Add bank_id column to DataFrame
df['bank_id'] = df['bank'].map(bank_id_map)



In [ ]:
# Rename 'date' to match DB schema
df = df.rename(columns={'date': 'review_date'})

# Extract relevant fields for insertion
records = df[['review', 'rating', 'review_date', 'bank_id', 'source', 'sentiment', 'sentiment_score']].values.tolist()


insert_query = """
    INSERT INTO reviews (review, rating, review_date, bank_id, source, sentiment, sentiment_score)
    VALUES %s;
"""

execute_values(cur, insert_query, records)
conn.commit()
print("✅ Data successfully inserted into PostgreSQL.")



In [ ]:
# Fetch and display first 5 rows from reviews table
cur.execute("""
    SELECT r.id, r.review, r.rating, r.review_date, b.name AS bank, r.source, r.sentiment, r.sentiment_score
    FROM reviews r
    JOIN banks b ON r.bank_id = b.id
    LIMIT 5;
""")

rows = cur.fetchall()

# Print results
for row in rows:
    print(row)

# Close connection
cur.close()
conn.close()